In [1]:
from torchvision import transforms, models
from torchvision.datasets import STL10
import numpy as np
import torch
import torch.nn as nn
import os
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from tqdm.notebook import tqdm_notebook
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from torchmetrics import F1Score
from torchvision.models import resnet18

In [44]:
preprocess =  transforms.Compose([
    transforms.ToTensor()
])

In [3]:
train_dataset = STL10("../data", split="train", download=True, transform=preprocess)

100%|██████████| 2640397119/2640397119 [04:10<00:00, 10533471.58it/s]


Extracting ../data\stl10_binary.tar.gz to ../data


In [4]:
test_dataset = STL10("../data", split="test", download=True, transform=preprocess)

Files already downloaded and verified


In [5]:
unlabeled_dataset = STL10("../data", split="unlabeled", download=True, transform=preprocess)

Files already downloaded and verified


In [6]:
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=64)
unlabeled_dataloader = DataLoader(unlabeled_dataset, batch_size=64, shuffle=True)

In [7]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DEVICE

'cuda'

In [8]:
torch.cuda.empty_cache()

In [9]:
class PretrainingModule(pl.LightningModule):
    def __init__(self, model, learning_rate, num_classes=3):
        super().__init__()
        self.model = model
        self.loss_fn = nn.CrossEntropyLoss()
        self.accuracy = F1Score(task="multiclass", average="macro", num_classes=num_classes)
        self.learning_rate = learning_rate

    def forward(self, text):
        return self.model(text)

    def training_step(self, batch, batch_idx):
        img, label = batch
        output = self(img)
        loss = self.loss_fn(output, img)
        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return loss

    def validation_step(self, batch, batch_idx):
        img, label = batch
        output = self(img)
        loss = self.loss_fn(output, img)
        self.log('val_loss', loss, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        return loss

    def test_step(self, batch, batch_idx):
        img, label = batch
        output = self(img)
        loss = self.loss_fn(output, img)
        self.log('test_loss', loss, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        return loss

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=self.learning_rate)
        # scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[200], gamma=0.1)
        # return {'optimizer': optimizer, 'lr_scheduler': scheduler}
        return optimizer

In [10]:
class TrainingModule(pl.LightningModule):
    def __init__(self, model, learning_rate, num_classes=3):
        super().__init__()
        self.model = model
        self.loss_fn = nn.CrossEntropyLoss()
        self.accuracy = F1Score(task="multiclass", average="macro", num_classes=num_classes)
        self.learning_rate = learning_rate

    def forward(self, text):
        return self.model(text)

    def training_step(self, batch, batch_idx):
        img, label = batch
        output = self(img)
        loss = self.loss_fn(output, label)
        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return loss

    def validation_step(self, batch, batch_idx):
        img, label = batch
        output = self(img)
        loss = self.loss_fn(output, label)
        self.log('val_loss', loss, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        return loss

    def test_step(self, batch, batch_idx):
        img, label = batch
        output = self(img)
        loss = self.loss_fn(output, label)
        self.log('test_loss', loss, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        return loss

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=self.learning_rate)
        # scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[200], gamma=0.1)
        # return {'optimizer': optimizer, 'lr_scheduler': scheduler}
        return optimizer


In [33]:
class STLDataModule(pl.LightningDataModule):
    def __init__(self, batch_size=32):
        super().__init__()
        self.batch_size = batch_size
        self.transform = transforms.Compose([
            transforms.Resize((32, 32)),
            transforms.ToTensor()
        ])

    def setup(self, stage=None):
        self.stl10_train = STL10(root='../data', split="train", download=True, transform=self.transform)
        self.stl10_test = STL10(root='../data', split="test", download=True, transform=self.transform)
        self.stl10_unlabeled = STL10(root='../data', split="unlabeled", download=True, transform=self.transform)

    def train_dataloader(self):
        return DataLoader(self.stl10_train, batch_size=self.batch_size, shuffle=True)

    def test_dataloader(self):
        return DataLoader(self.stl10_test, batch_size=self.batch_size, shuffle=False)
    
    def unlabeled_dataloader(self):
        return DataLoader(self.stl10_unlabeled, batch_size=self.batch_size, shuffle=False)


In [12]:
class BasicBlock(nn.Module):
    expansion = 1

    def __main__(self, in_channels, out_channels, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out

class ResNet18(nn.Module):
    def __init__(self):
        super().__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.layer1 = self._make_layer(64, 2, stride=1)
        self.layer2 = self._make_layer(128, 2, stride=2)
        self.layer3 = self._make_layer(256, 2, stride=2)
        self.layer4 = self._make_layer(512, 2, stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * BasicBlock.expansion, 1000)

    def _make_layer(self, out_channels, blocks, stride):
        downsample = None
        if stride != 1 or self.in_channels != out_channels * BasicBlock.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.in_channels, out_channels * BasicBlock.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels * BasicBlock.expansion),
            )

        layers = []
        layers.append(BasicBlock(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels * BasicBlock.expansion
        for _ in range(1, blocks):
            layers.append(BasicBlock(self.in_channels, out_channels))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

In [13]:
# resnet = ResNet18()

In [14]:
import torchvision
import os

class ResNetEncoder(nn.Module):
    def __init__(self):
        super(ResNetEncoder, self).__init__()
        resnet = models.resnet18(pretrained=True)
        self.resnet = nn.Sequential(*list(resnet.children())[:-1])
        self.fc = nn.Linear(resnet.fc.in_features, 100)

    def forward(self, x):
        with torch.no_grad():
            x = self.resnet(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.fc = nn.Linear(100, 256 * 4 * 4)
        self.deconv1 = nn.ConvTranspose2d(256, 128, 4, 2, 1)
        self.deconv2 = nn.ConvTranspose2d(128, 64, 4, 2, 1)
        self.deconv3 = nn.ConvTranspose2d(64, 3, 4, 2, 1)

    def forward(self, z):
        z = F.relu(self.fc(z))
        z = z.view(z.size(0), 256, 4, 4)
        z = F.relu(self.deconv1(z))
        z = F.relu(self.deconv2(z))
        z = torch.tanh(self.deconv3(z))
        return z

class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=4, stride=2, padding=1)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1)
        self.fc1 = nn.Linear(256 * 4 * 4, 512)
        self.fc2 = nn.Linear(512 + 100, 1)

    def forward(self, x, z):
        x = F.leaky_relu(self.conv1(x), 0.2)
        x = F.leaky_relu(self.conv2(x), 0.2)
        x = F.leaky_relu(self.conv3(x), 0.2)
        x = x.view(x.size(0), -1)
        x = F.leaky_relu(self.fc1(x), 0.2)
        x = torch.cat([x, z], dim=1)
        x = torch.sigmoid(self.fc2(x))
        return x

class BiGAN(pl.LightningModule):
    def __init__(self):
        super(BiGAN, self).__init__()
        self.encoder = ResNetEncoder()
        self.generator = Generator()
        self.discriminator = Discriminator()
        self.criterion = nn.BCELoss()
        self.automatic_optimization = False
    
    def forward(self, z):
        return self.generator(z)
    
    def configure_optimizers(self):
        lr = 0.0002
        beta1 = 0.5
        optimizerE = optim.Adam(self.encoder.parameters(), lr=lr, betas=(beta1, 0.999))
        optimizerG = optim.Adam(self.generator.parameters(), lr=lr, betas=(beta1, 0.999))
        optimizerD = optim.Adam(self.discriminator.parameters(), lr=lr, betas=(beta1, 0.999))
        return [optimizerE, optimizerG, optimizerD]
    
    def training_step(self, batch, batch_idx):
        images, _ = batch
        batch_size = images.size(0)
        real_labels = torch.ones(batch_size, 1).type_as(images)
        fake_labels = torch.zeros(batch_size, 1).type_as(images)
        
        optE, optG, optD = self.optimizers()

        z = torch.randn(batch_size, 100).type_as(images)
        fake_images = self.generator(z)
        real_outputs = self.discriminator(images, self.encoder(images))
        fake_outputs = self.discriminator(fake_images, z)
        d_loss_real = self.criterion(real_outputs, real_labels)
        d_loss_fake = self.criterion(fake_outputs, fake_labels)
        d_loss = d_loss_real + d_loss_fake
        
        optD.zero_grad()
        self.manual_backward(d_loss, retain_graph=True)
        optD.step()

        fake_outputs = self.discriminator(fake_images, z)
        g_loss = self.criterion(fake_outputs, real_labels)
        
        optG.zero_grad()
        optE.zero_grad()
        self.manual_backward(g_loss)
        optG.step()
        optE.step()
        
        self.log('d_loss', d_loss, prog_bar=True)
        self.log('g_loss', g_loss, prog_bar=True)
        
        return {'d_loss': d_loss, 'g_loss': g_loss}

In [45]:
stl10_dm = STLDataModule(batch_size=32)
stl10_dm.setup()

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [16]:
bigan_model = BiGAN()
checkpoint_callback = ModelCheckpoint(
    monitor='d_loss',
    dirpath='./checkpoints',
    filename='bigan-{epoch:02d}-{d_loss:.2f}',
    save_top_k=3,
    mode='min',
)

trainer = pl.Trainer(
    max_epochs=100,
    callbacks=[checkpoint_callback]
)
trainer.fit(bigan_model, stl10_dm)

c:\Users\Adam\miniconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Adam\miniconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
c:\Users\Adam\miniconda3\Lib\site-packages\pytorch_lightning\trainer\connectors\logger_connector\logger_connector.py:75: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts w

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


c:\Users\Adam\miniconda3\Lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:652: Checkpoint directory C:\Users\Adam\Desktop\PGM_UR\representation-learning-methods\notebooks\checkpoints exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type          | Params | Mode 
--------------------------------------------------------
0 | encoder       | ResNetEncoder | 11.2 M | train
1 | generator     | Generator     | 1.1 M  | train
2 | discriminator | Discriminator | 2.8 M  | train
3 | criterion     | BCELoss       | 0      | train
--------------------------------------------------------
15.1 M    Trainable params
0         Non-trainable params
15.1 M    Total params
60.229    Total estimated model params size (MB)
c:\Users\Adam\miniconda3\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` ar

Epoch 0:   0%|          | 0/157 [00:00<?, ?it/s] 

c:\Users\Adam\miniconda3\Lib\site-packages\torch\nn\modules\conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\cudnn\Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


Epoch 99: 100%|██████████| 157/157 [00:04<00:00, 34.77it/s, v_num=47, d_loss=0.0768, g_loss=10.30] 

`Trainer.fit` stopped: `max_epochs=100` reached.


Epoch 99: 100%|██████████| 157/157 [00:04<00:00, 34.75it/s, v_num=47, d_loss=0.0768, g_loss=10.30]
